In [1]:
import json
import src.utils as u
import numpy as np
import pandas as pd
from pymongo import MongoClient

In [2]:
client = MongoClient("mongodb://localhost/companies")
db = client.get_database()

## Filtamos por documentos que tengan los campos funding_round y offices no vacio, que sus oficinas tengan latitud y longitud y además la compañía no hayan quebrado.

In [3]:
companies = list(db["companies"].find({"funding_rounds":{"$exists": True, "$not": {"$size": 0}},"offices":{"$exists": True, "$not": {"$size": 0}}, "offices.latitude":{"$exists":True, "$ne":None}, "offices.longitude":{"$exists":True, "$ne":None},"deadpooled_year":{"$not":{"$gte":0}}},{"name":1,"offices":1,"category_code":1, "funding_rounds":1, "founded_year":1}))
df = pd.DataFrame(companies)
df.head()

,_id,name,category_code,founded_year,funding_rounds,offices
0,52cdef7c4bab8bd675297d8d,Digg,news,2004.0,"[{'id': 1, 'round_code': 'b', 'source_url': 'h...","[{'description': None, 'address1': '135 Missis..."
1,52cdef7c4bab8bd675297d91,Geni,web,2006.0,"[{'id': 6, 'round_code': 'a', 'source_url': ''...","[{'description': 'Headquarters', 'address1': '..."
2,52cdef7c4bab8bd675297d96,Gizmoz,web,2003.0,"[{'id': 9, 'round_code': 'a', 'source_url': 'h...","[{'description': None, 'address1': None, 'addr..."
3,52cdef7c4bab8bd675297d97,Scribd,news,2007.0,"[{'id': 10, 'round_code': 'seed', 'source_url'...","[{'description': 'HQ', 'address1': '539 Bryant..."
4,52cdef7c4bab8bd675297d99,Lala,games_video,NaN,"[{'id': 16, 'round_code': 'b', 'source_url': '...","[{'description': 'Lala Headquarters', 'address..."


## Calculamos la suma total de las funding rounds y devolvemos el total en USD en la columna "Total_funding"

In [4]:
total_funding = u.sum_funding_rounds(df.funding_rounds)
df["Total_funding"] = total_funding
df.head()

,_id,name,category_code,founded_year,funding_rounds,offices,Total_funding
0,52cdef7c4bab8bd675297d8d,Digg,news,2004.0,"[{'id': 1, 'round_code': 'b', 'source_url': 'h...","[{'description': None, 'address1': '135 Missis...",45000000.0
1,52cdef7c4bab8bd675297d91,Geni,web,2006.0,"[{'id': 6, 'round_code': 'a', 'source_url': ''...","[{'description': 'Headquarters', 'address1': '...",16500000.0
2,52cdef7c4bab8bd675297d96,Gizmoz,web,2003.0,"[{'id': 9, 'round_code': 'a', 'source_url': 'h...","[{'description': None, 'address1': None, 'addr...",18100000.0
3,52cdef7c4bab8bd675297d97,Scribd,news,2007.0,"[{'id': 10, 'round_code': 'seed', 'source_url'...","[{'description': 'HQ', 'address1': '539 Bryant...",25762000.0
4,52cdef7c4bab8bd675297d99,Lala,games_video,NaN,"[{'id': 16, 'round_code': 'b', 'source_url': '...","[{'description': 'Lala Headquarters', 'address...",44150000.0


## Sacamos las oficinas de cada compañia una por fila y sus campos pasaran a las columnas

In [5]:
df = df.explode('offices')
df_offices= df[["offices"]].apply(lambda r: r.offices, result_type="expand", axis=1)
df = pd.concat([df,df_offices], axis=1)
df.head()

,_id,name,category_code,founded_year,funding_rounds,offices,Total_funding,description,address1,address2,zip_code,city,state_code,country_code,latitude,longitude
0,52cdef7c4bab8bd675297d8d,Digg,news,2004.0,"[{'id': 1, 'round_code': 'b', 'source_url': 'h...","{'description': None, 'address1': '135 Mississ...",45000000.0,None,135 Mississippi St,,94107,San Francisco,CA,USA,37.764726,-122.394523
1,52cdef7c4bab8bd675297d91,Geni,web,2006.0,"[{'id': 6, 'round_code': 'a', 'source_url': ''...","{'description': 'Headquarters', 'address1': '9...",16500000.0,Headquarters,9229 W. Sunset Blvd.,,90069,West Hollywood,CA,USA,34.090368,-118.393064
2,52cdef7c4bab8bd675297d96,Gizmoz,web,2003.0,"[{'id': 9, 'round_code': 'a', 'source_url': 'h...","{'description': None, 'address1': None, 'addre...",18100000.0,None,None,None,None,Menlo Park,CA,USA,37.484130,-122.169472
3,52cdef7c4bab8bd675297d97,Scribd,news,2007.0,"[{'id': 10, 'round_code': 'seed', 'source_url'...","{'description': 'HQ', 'address1': '539 Bryant ...",25762000.0,HQ,539 Bryant Street,,94107,San Francisco,CA,USA,37.789634,-122.404052
4,52cdef7c4bab8bd675297d99,Lala,games_video,NaN,"[{'id': 16, 'round_code': 'b', 'source_url': '...","{'description': 'Lala Headquarters', 'address1...",44150000.0,Lala Headquarters,209 Hamilton Ave,Suite #200,94301,Palo Alto,CA,USA,37.451151,-122.154369


In [6]:
df = df.drop(columns = ["offices","funding_rounds", "description", "address2", "state_code", "country_code"])

## Creamos la columna location como un diccionaro de tipo point que nos permita lanzar geoquerys contra la bbdd

In [7]:
df["location"] = df[["latitude","longitude"]].apply(lambda x:u.asGeoJSON(x.latitude,x.longitude), axis=1)
df = df.drop(columns=["_id","address1","zip_code"])
df.head()

,name,category_code,founded_year,Total_funding,city,latitude,longitude,location
0,Digg,news,2004.0,45000000.0,San Francisco,37.764726,-122.394523,"{'type': 'Point', 'coordinates': [-122.394523,..."
1,Geni,web,2006.0,16500000.0,West Hollywood,34.090368,-118.393064,"{'type': 'Point', 'coordinates': [-118.393064,..."
2,Gizmoz,web,2003.0,18100000.0,Menlo Park,37.484130,-122.169472,"{'type': 'Point', 'coordinates': [-122.169472,..."
3,Scribd,news,2007.0,25762000.0,San Francisco,37.789634,-122.404052,"{'type': 'Point', 'coordinates': [-122.404052,..."
4,Lala,games_video,NaN,44150000.0,Palo Alto,37.451151,-122.154369,"{'type': 'Point', 'coordinates': [-122.154369,..."


## Creamos un json que almacenamos en la carpeta OUTPUT y creamos un nuevo documento en la bbdd companies que se llama companies clean y tiene un indice del tipo 2dsphere (para realizar geoquerys)

In [8]:
df.to_json("OUTPUT/cleaned_offices.json", orient="records")
u.create_document("companies", 'OUTPUT/cleaned_offices.json', 'companies_clean', True)

## Preparamos una bbdd de aeropuertos con el csv  que se encuentra en INPUT/airport-codes_csv.csv:

In [9]:
airports = pd.read_csv("INPUT/airport-codes_csv.csv")
airports = airports.drop(columns=["ident", "gps_code", "local_code"])

## Creamos la columna location para las geoquerys y las columnas latitude y longitude que las usaremos más adelante para calcular distancias.

In [10]:
airports["location"] = airports["coordinates"].apply(lambda x:u.asGeoJSON(float(x.split(",")[1].strip()),float(x.split(",")[0].strip())))
airports["longitude"] = airports.coordinates.apply(lambda x: float(x.split(",")[0].strip()))
airports["latitude"] = airports.coordinates.apply(lambda x: float(x.split(",")[1].strip()))
airports = airports.drop(columns=["coordinates","elevation_ft", "continent", "iso_country", "iso_region"])
airports.head()

,type,name,municipality,iata_code,location,longitude,latitude
0,heliport,Total Rf Heliport,Bensalem,NaN,"{'type': 'Point', 'coordinates': [-74.93360137...",-74.933601,40.070801
1,small_airport,Aero B Ranch Airport,Leoti,NaN,"{'type': 'Point', 'coordinates': [-101.473911,...",-101.473911,38.704022
2,small_airport,Lowell Field,Anchor Point,NaN,"{'type': 'Point', 'coordinates': [-151.6959991...",-151.695999,59.949200
3,small_airport,Epps Airpark,Harvest,NaN,"{'type': 'Point', 'coordinates': [-86.77030181...",-86.770302,34.864799
4,closed,Newport Hospital & Clinic Heliport,Newport,NaN,"{'type': 'Point', 'coordinates': [-91.254898, ...",-91.254898,35.608700


## Creamos el documento

In [11]:
airports.to_json("OUTPUT/cleaned_airports.json", orient="records")
u.create_document("airports", 'OUTPUT/cleaned_airports.json', 'cleaned_airports', True)

## Vamos a quedarnos solo con los aeropuertos internacionales, nos es más fácil conseguir los valores de iata validos a través de una consulta pymongo.
## Sobreescribimos la base de datos para tenerla con los datos que queremos

In [12]:
client = MongoClient("mongodb://localhost/airports")
db = client.get_database()
ar = list(db["cleaned_airports"].find({"iata_code":{"$exists": True, "$ne": None}}))
df_ar = pd.DataFrame(ar)
df_ar = df_ar.drop(columns="_id")
df_ar.to_json("OUTPUT/cleaned_airports.json", orient="records")
u.create_document("airports", 'OUTPUT/cleaned_airports.json', 'cleaned_airports', True)

## Buscamos las empresas que se dedican al diseño:

In [13]:
df_design = df[df.category_code == "design"]
df_design

,name,category_code,founded_year,Total_funding,city,latitude,longitude,location
628,99designs,design,2008.0,35000000.0,San Francisco,37.795531,-122.400598,"{'type': 'Point', 'coordinates': [-122.4005983..."
628,99designs,design,2008.0,35000000.0,Collingwood,-37.802659,144.986855,"{'type': 'Point', 'coordinates': [144.9868546,..."
628,99designs,design,2008.0,35000000.0,Berlin,52.498620,13.446903,"{'type': 'Point', 'coordinates': [13.4469031, ..."


## Con nuestros filtros sólo hay 3, como queremos que esten cerca de empresas que se dedican a diseño buscaremos el resto de los servicios en Berlín:

### Guarderías para los empleados con hijos:

In [14]:
kindergarten = u.get_coordinates("Berlin", services=["nursery school"])
kindergarten.head()

,id,name,contact,location,categories,verified,stats,beenHere,photos,hereNow,venuePage
0,4cdfe764ffcf370470a81c82,INA.KINDER.GARTEN Dresdener Straße,{},"{'address': 'Dresdener Straße 128', 'lat': 52....","[{'id': '4f4533814b9074f6e4fb0107', 'name': 'N...",False,"{'tipCount': 0, 'usersCount': 0, 'checkinsCoun...","{'count': 0, 'lastCheckinExpiredAt': 0, 'marke...","{'count': 0, 'groups': []}","{'count': 0, 'summary': 'Nobody here', 'groups...",NaN
1,52385ad011d2790ab72b2582,Kindergärten City,{},"{'lat': 52.50045546510848, 'lng': 13.443106804...","[{'id': '4f4533814b9074f6e4fb0107', 'name': 'N...",False,"{'tipCount': 0, 'usersCount': 0, 'checkinsCoun...","{'count': 0, 'lastCheckinExpiredAt': 0, 'marke...","{'count': 0, 'groups': []}","{'count': 0, 'summary': 'Nobody here', 'groups...",NaN
2,51ff4f8a498e883b01f44c10,Kindergarten Kleine Auguststraße,{},"{'address': 'Kleine Auguststr. 10-12', 'lat': ...","[{'id': '4f4533814b9074f6e4fb0107', 'name': 'N...",False,"{'tipCount': 0, 'usersCount': 0, 'checkinsCoun...","{'count': 0, 'lastCheckinExpiredAt': 0, 'marke...","{'count': 0, 'groups': []}","{'count': 0, 'summary': 'Nobody here', 'groups...",NaN
3,4ecbd725f790bc9a81a32e72,BCS Kindergarten in the Park,{},"{'address': 'Invalidenstr. 130', 'lat': 52.531...","[{'id': '4f4533814b9074f6e4fb0107', 'name': 'N...",False,"{'tipCount': 0, 'usersCount': 0, 'checkinsCoun...","{'count': 0, 'lastCheckinExpiredAt': 0, 'marke...","{'count': 0, 'groups': []}","{'count': 0, 'summary': 'Nobody here', 'groups...",NaN
4,5a11ab4a6a59503862e588f3,Wunderhaus,{},"{'lat': 52.53271959459243, 'lng': 13.416940178...","[{'id': '4f4533814b9074f6e4fb0107', 'name': 'N...",False,"{'tipCount': 0, 'usersCount': 0, 'checkinsCoun...","{'count': 0, 'lastCheckinExpiredAt': 0, 'marke...","{'count': 0, 'groups': []}","{'count': 0, 'summary': 'Nobody here', 'groups...",NaN


### Restaurantes vanganos para el CEO:

In [15]:
vegan = u.get_coordinates("Berlin", services=["vegan"])
vegan.head()

,id,name,contact,location,categories,verified,stats,beenHere,photos,hereNow,venuePage
0,5515ba58498e626740d4f760,Alaska,{},"{'address': 'Reuterstr. 85', 'lat': 52.4828751...","[{'id': '4bf58dd8d48988d1d3941735', 'name': 'V...",False,"{'tipCount': 0, 'usersCount': 0, 'checkinsCoun...","{'count': 0, 'lastCheckinExpiredAt': 0, 'marke...","{'count': 0, 'groups': []}","{'count': 0, 'summary': 'Nobody here', 'groups...",NaN
1,58669dfd13af1c32d09c0a44,1990 Vegan Living,{},"{'address': 'Krossener Str. 19', 'crossStreet'...","[{'id': '4bf58dd8d48988d1d3941735', 'name': 'V...",False,"{'tipCount': 0, 'usersCount': 0, 'checkinsCoun...","{'count': 0, 'lastCheckinExpiredAt': 0, 'marke...","{'count': 0, 'groups': []}","{'count': 0, 'summary': 'Nobody here', 'groups...",NaN
2,4ade320cf964a520cf7321e3,W - Der Imbiss,{},"{'address': 'Kastanienallee 49', 'crossStreet'...","[{'id': '4bf58dd8d48988d1d3941735', 'name': 'V...",True,"{'tipCount': 0, 'usersCount': 0, 'checkinsCoun...","{'count': 0, 'lastCheckinExpiredAt': 0, 'marke...","{'count': 0, 'groups': []}","{'count': 0, 'summary': 'Nobody here', 'groups...",{'id': '469207190'}
3,55608437498e157f858c6eec,SOY,{},"{'address': 'Rosa-Luxemburg-Str. 30', 'lat': 5...","[{'id': '4bf58dd8d48988d1d3941735', 'name': 'V...",False,"{'tipCount': 0, 'usersCount': 0, 'checkinsCoun...","{'count': 0, 'lastCheckinExpiredAt': 0, 'marke...","{'count': 0, 'groups': []}","{'count': 0, 'summary': 'Nobody here', 'groups...",NaN
4,4c34606a7cc0c9b68c2af39a,Café Vux,{},"{'address': 'Wipperstr. 14', 'crossStreet': 'K...","[{'id': '4bf58dd8d48988d1d3941735', 'name': 'V...",False,"{'tipCount': 0, 'usersCount': 0, 'checkinsCoun...","{'count': 0, 'lastCheckinExpiredAt': 0, 'marke...","{'count': 0, 'groups': []}","{'count': 0, 'summary': 'Nobody here', 'groups...",NaN


### Bares para salir

In [16]:
bar = u.get_coordinates("Berlin", services=["bar"])
bar.head()

,id,name,contact,location,categories,verified,stats,beenHere,photos,hereNow,venuePage
0,4adcda79f964a520b44621e3,Luzia,{},"{'address': 'Oranienstr. 34', 'lat': 52.501607...","[{'id': '4bf58dd8d48988d116941735', 'name': 'B...",False,"{'tipCount': 0, 'usersCount': 0, 'checkinsCoun...","{'count': 0, 'lastCheckinExpiredAt': 0, 'marke...","{'count': 0, 'groups': []}","{'count': 0, 'summary': 'Nobody here', 'groups...",NaN
1,4adcda7af964a520ea4621e3,mama bar,{},"{'address': 'Hobrechtstr. 61', 'lat': 52.49054...","[{'id': '4bf58dd8d48988d116941735', 'name': 'B...",False,"{'tipCount': 0, 'usersCount': 0, 'checkinsCoun...","{'count': 0, 'lastCheckinExpiredAt': 0, 'marke...","{'count': 0, 'groups': []}","{'count': 0, 'summary': 'Nobody here', 'groups...",NaN
2,4fa43face4b0bcbb46297a7d,Fitcher’s Vogel,{},"{'address': 'Warschauer Str. 26', 'lat': 52.51...","[{'id': '4bf58dd8d48988d116941735', 'name': 'B...",False,"{'tipCount': 0, 'usersCount': 0, 'checkinsCoun...","{'count': 0, 'lastCheckinExpiredAt': 0, 'marke...","{'count': 0, 'groups': []}","{'count': 0, 'summary': 'Nobody here', 'groups...",NaN
3,4adf6269f964a5201d7a21e3,Fuchsbau,{},"{'address': 'Planufer 93-95', 'lat': 52.495781...","[{'id': '4bf58dd8d48988d116941735', 'name': 'B...",False,"{'tipCount': 0, 'usersCount': 0, 'checkinsCoun...","{'count': 0, 'lastCheckinExpiredAt': 0, 'marke...","{'count': 0, 'groups': []}","{'count': 0, 'summary': 'Nobody here', 'groups...",NaN
4,4bb5002129269c74a9f1ca92,Minimal Bar,{},"{'address': 'Rigaer Str. 31', 'crossStreet': '...","[{'id': '4bf58dd8d48988d116941735', 'name': 'B...",False,"{'tipCount': 0, 'usersCount': 0, 'checkinsCoun...","{'count': 0, 'lastCheckinExpiredAt': 0, 'marke...","{'count': 0, 'groups': []}","{'count': 0, 'summary': 'Nobody here', 'groups...",NaN


### Starbucks

In [17]:
starbucks = u.get_coordinates("Berlin", services=["starbucks"])
starbucks.head()

,id,name,contact,location,categories,verified,stats,beenHere,photos,hereNow
0,559e7c48498e877f08540314,Starbucks,{},"{'address': 'Schloßstr. 129', 'crossStreet': '...","[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",False,"{'tipCount': 0, 'usersCount': 0, 'checkinsCoun...","{'count': 0, 'lastCheckinExpiredAt': 0, 'marke...","{'count': 0, 'groups': []}","{'count': 0, 'summary': 'Nobody here', 'groups..."
1,4bb37c8ba32876b0dac001fe,Starbucks,{},"{'address': 'Potsdamer Platz 5', 'crossStreet'...","[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",False,"{'tipCount': 0, 'usersCount': 0, 'checkinsCoun...","{'count': 0, 'lastCheckinExpiredAt': 0, 'marke...","{'count': 0, 'groups': []}","{'count': 0, 'summary': 'Nobody here', 'groups..."
2,4b61c1fff964a520e1202ae3,Starbucks,{},"{'address': 'Kurfürstendamm 61', 'lat': 52.500...","[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",False,"{'tipCount': 0, 'usersCount': 0, 'checkinsCoun...","{'count': 0, 'lastCheckinExpiredAt': 0, 'marke...","{'count': 0, 'groups': []}","{'count': 0, 'summary': 'Nobody here', 'groups..."
3,5c225b1da35dce002cbd3da8,Starbucks,{},"{'address': 'Joachimsthaler Str. 3', 'lat': 52...","[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",False,"{'tipCount': 0, 'usersCount': 0, 'checkinsCoun...","{'count': 0, 'lastCheckinExpiredAt': 0, 'marke...","{'count': 0, 'groups': []}","{'count': 0, 'summary': 'Nobody here', 'groups..."
4,4bd2ebc477b29c74dd888f82,Starbucks,{},"{'address': 'Kurfürstendamm 231', 'lat': 52.50...","[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",False,"{'tipCount': 0, 'usersCount': 0, 'checkinsCoun...","{'count': 0, 'lastCheckinExpiredAt': 0, 'marke...","{'count': 0, 'groups': []}","{'count': 0, 'summary': 'Nobody here', 'groups..."


## Vamos a coger los aeropuertos internacionales más grandes de Berlin

In [18]:
arp_berlin = df_ar[df_ar.municipality == "Berlin"]
arp_berlin = arp_berlin[arp_berlin["type"] == "large_airport"].reset_index()
arp_berlin

,index,type,name,municipality,iata_code,location,longitude,latitude
0,1245,large_airport,Berlin-SchÃ¶nefeld Airport,Berlin,SXF,"{'type': 'Point', 'coordinates': [13.5225, 52....",13.5225,52.380001
1,1259,large_airport,Berlin-Tegel Airport,Berlin,TXL,"{'type': 'Point', 'coordinates': [13.2877, 52....",13.2877,52.559700


## Nos traemos las empresas que estan en Berlí en un radio de 30km

In [19]:
query = u.withGeoQuery("Berlin", 30000)
client = MongoClient("mongodb://localhost/companies")
db = client.get_database()
comp = list(db["companies_clean"].find(query))
companies_berlin = pd.DataFrame(comp)
companies_berlin.head()

,_id,name,category_code,founded_year,Total_funding,city,latitude,longitude,location
0,5e4179ba353fb003e6d63e92,Moviepilot,web,2007.0,7000000.0,Berlin,52.494727,13.398504,"{'type': 'Point', 'coordinates': [13.398504, 5..."
1,5e4179ba353fb003e6d6379d,mokono,advertising,2006.0,0.0,Berlin,52.500888,13.420817,"{'type': 'Point', 'coordinates': [13.420817, 5..."
2,5e4179ba353fb003e6d63697,alaTest,web,2005.0,0.0,Berlin,52.509299,13.390060,"{'type': 'Point', 'coordinates': [13.3900596, ..."
3,5e4179ba353fb003e6d63db8,My-Hammer,web,2005.0,3200000.0,Berlin,52.509231,13.388361,"{'type': 'Point', 'coordinates': [13.3883606, ..."
4,5e4179ba353fb003e6d63b70,Kaeuferportal,enterprise,2008.0,0.0,Berlin,52.512337,13.393667,"{'type': 'Point', 'coordinates': [13.3936665, ..."


## Vamos a coger las que han conseguido más de 1M en las rondas de financiación que es de las que queremos que nos rodeen:

In [20]:
companies_berlin_1M = companies_berlin[companies_berlin.Total_funding > 1000000].reset_index()
companies_berlin_1M.head()

,index,_id,name,category_code,founded_year,Total_funding,city,latitude,longitude,location
0,0,5e4179ba353fb003e6d63e92,Moviepilot,web,2007.0,7000000.0,Berlin,52.494727,13.398504,"{'type': 'Point', 'coordinates': [13.398504, 5..."
1,3,5e4179ba353fb003e6d63db8,My-Hammer,web,2005.0,3200000.0,Berlin,52.509231,13.388361,"{'type': 'Point', 'coordinates': [13.3883606, ..."
2,5,5e4179ba353fb003e6d63667,Metaversum,web,2006.0,6260000.0,Berlin,52.512249,13.419841,"{'type': 'Point', 'coordinates': [13.419841, 5..."
3,6,5e4179ba353fb003e6d63643,babbel,education,2007.0,10000000.0,Berlin,52.489700,13.388610,"{'type': 'Point', 'coordinates': [13.38861, 52..."
4,11,5e4179ba353fb003e6d634a0,Plazes,web,2006.0,4033000.0,Berlin,52.523402,13.398694,"{'type': 'Point', 'coordinates': [13.398694, 5..."


## Además no queremos que haya empresas de más de 10 años en un radio de 2 km, así que también las tenemos que tener en cuenta:

In [21]:
companies_berlin_10_years = companies_berlin[companies_berlin.founded_year < 2003].reset_index()
companies_berlin_10_years.head()

,index,_id,name,category_code,founded_year,Total_funding,city,latitude,longitude,location
0,12,5e4179ba353fb003e6d6375d,gate5,mobile,1999.0,12535000.0,Berlin,52.523480,13.411494,"{'type': 'Point', 'coordinates': [13.411494, 5..."
1,29,5e4179ba353fb003e6d63d7e,MergeOptics,public_relations,2000.0,8260000.0,Berlin,52.583651,13.285504,"{'type': 'Point', 'coordinates': [13.2855038, ..."


## Ahora calcularemos la distancia que hay a la oficina que se dedica al diseño que, al ser única, será la que usaremos para como base para localizar nuestra oficina:

In [22]:
design = companies_berlin[companies_berlin.category_code=="design"].reset_index()

In [23]:
u.calculate_distance(design, [arp_berlin, companies_berlin_1M, companies_berlin_10_years])
u.calculate_service_distance(design, [kindergarten, starbucks, vegan, bar])

In [24]:
vegan = vegan.sort_values(["distance"]).reset_index()
starbucks = starbucks.sort_values(["distance"]).reset_index()
bar = bar.sort_values(["distance"]).reset_index()
companies_berlin_1M = companies_berlin_1M.sort_values(["distance"]).reset_index()
arp_berlin = arp_berlin.sort_values(["distance"]).reset_index()
kindergarten = kindergarten.sort_values(["distance"]).reset_index()

In [25]:
vegan.sort_values("distance").head(2)
kindergarten.sort_values("distance").head(2)
bar.sort_values("distance").head(2)
starbucks.sort_values("distance").head(2)
companies_berlin_1M.sort_values("distance").head(2)
arp_berlin.sort_values("distance").head(2)

,level_0,index,type,name,municipality,iata_code,location,longitude,latitude,distance
0,1,1259,large_airport,Berlin-Tegel Airport,Berlin,TXL,"{'type': 'Point', 'coordinates': [13.2877, 52....",13.2877,52.559700,12.764448
1,0,1245,large_airport,Berlin-SchÃ¶nefeld Airport,Berlin,SXF,"{'type': 'Point', 'coordinates': [13.5225, 52....",13.5225,52.380001,14.165190


## Ahora calcularemos el punto en el que queremos colocar nuestra oficina en función de los servicios y oficinas que queremos que esten cerca.
## Lo hacemos calculando el punto medio entre los servicios más cercanos

In [26]:
office = u.calculate_point([vegan, bar, kindergarten, starbucks], [companies_berlin_1M])
office = {"latitude": [office[0]], "longitude": [office[1]]}

## Calculamos de nuevo las distancias en base a la localización de la oficina

In [27]:
u.calculate_distance(office, [arp_berlin, companies_berlin_1M, companies_berlin_10_years])
u.calculate_service_distance(office, [kindergarten, starbucks, vegan, bar])

## Ambas a más de 2km

In [28]:
companies_berlin_10_years.sort_values("distance")

,index,_id,name,category_code,founded_year,Total_funding,city,latitude,longitude,location,distance
0,12,5e4179ba353fb003e6d6375d,gate5,mobile,1999.0,12535000.0,Berlin,52.523480,13.411494,"{'type': 'Point', 'coordinates': [13.411494, 5...",2.127246
1,29,5e4179ba353fb003e6d63d7e,MergeOptics,public_relations,2000.0,8260000.0,Berlin,52.583651,13.285504,"{'type': 'Point', 'coordinates': [13.2855038, ...",12.578643


In [29]:
kindergarten.sort_values("distance").head(2)

,index,id,name,contact,location,categories,verified,stats,beenHere,photos,hereNow,venuePage,distance
2,0,4cdfe764ffcf370470a81c82,INA.KINDER.GARTEN Dresdener Straße,{},"{'address': 'Dresdener Straße 128', 'lat': 52....","[{'id': '4f4533814b9074f6e4fb0107', 'name': 'N...",False,"{'tipCount': 0, 'usersCount': 0, 'checkinsCoun...","{'count': 0, 'lastCheckinExpiredAt': 0, 'marke...","{'count': 0, 'groups': []}","{'count': 0, 'summary': 'Nobody here', 'groups...",NaN,0.513891
0,1,52385ad011d2790ab72b2582,Kindergärten City,{},"{'lat': 52.50045546510848, 'lng': 13.443106804...","[{'id': '4f4533814b9074f6e4fb0107', 'name': 'N...",False,"{'tipCount': 0, 'usersCount': 0, 'checkinsCoun...","{'count': 0, 'lastCheckinExpiredAt': 0, 'marke...","{'count': 0, 'groups': []}","{'count': 0, 'summary': 'Nobody here', 'groups...",NaN,1.746292


In [30]:
vegan.sort_values("distance").head(2)

,index,id,name,contact,location,categories,verified,stats,beenHere,photos,hereNow,venuePage,distance
2,9,525bcc3f11d2b20ff55820e8,no milk today,{},"{'address': 'Fichtestr. 3', 'lat': 52.49097630...","[{'id': '4bf58dd8d48988d1d3941735', 'name': 'V...",True,"{'tipCount': 0, 'usersCount': 0, 'checkinsCoun...","{'count': 0, 'lastCheckinExpiredAt': 0, 'marke...","{'count': 0, 'groups': []}","{'count': 0, 'summary': 'Nobody here', 'groups...",{'id': '80731135'},1.585081
5,6,55031515498ecb9c699ed0a0,Café 93,{},"{'address': 'Monbijouplatz 2', 'lat': 52.52299...","[{'id': '4bf58dd8d48988d1d3941735', 'name': 'V...",False,"{'tipCount': 0, 'usersCount': 0, 'checkinsCoun...","{'count': 0, 'lastCheckinExpiredAt': 0, 'marke...","{'count': 0, 'groups': []}","{'count': 0, 'summary': 'Nobody here', 'groups...",NaN,2.385231


In [31]:
starbucks.sort_values("distance").head(2)

,index,id,name,contact,location,categories,verified,stats,beenHere,photos,hereNow,distance
0,5,4adf768af964a520d47a21e3,Starbucks,{},"{'address': 'Friedrichstr. 61', 'crossStreet':...","[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",False,"{'tipCount': 0, 'usersCount': 0, 'checkinsCoun...","{'count': 0, 'lastCheckinExpiredAt': 0, 'marke...","{'count': 0, 'groups': []}","{'count': 0, 'summary': 'Nobody here', 'groups...",2.054384
1,9,4adf77c3f964a520df7a21e3,Starbucks,{},"{'address': 'Friedrichstr. 96', 'crossStreet':...","[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",False,"{'tipCount': 0, 'usersCount': 0, 'checkinsCoun...","{'count': 0, 'lastCheckinExpiredAt': 0, 'marke...","{'count': 0, 'groups': []}","{'count': 0, 'summary': 'Nobody here', 'groups...",2.590451


In [32]:
arp_berlin.sort_values("distance").head()

,level_0,index,type,name,municipality,iata_code,location,longitude,latitude,distance
0,1,1259,large_airport,Berlin-Tegel Airport,Berlin,TXL,"{'type': 'Point', 'coordinates': [13.2877, 52....",13.2877,52.559700,10.768203
1,0,1245,large_airport,Berlin-SchÃ¶nefeld Airport,Berlin,SXF,"{'type': 'Point', 'coordinates': [13.5225, 52....",13.5225,52.380001,15.590253


In [33]:
companies_berlin_1M.sort_values("distance").head(3)

,level_0,index,_id,name,category_code,founded_year,Total_funding,city,latitude,longitude,location,distance
1,2,5,5e4179ba353fb003e6d63667,Metaversum,web,2006.0,6260000.0,Berlin,52.512249,13.419841,"{'type': 'Point', 'coordinates': [13.419841, 5...",0.830678
2,0,0,5e4179ba353fb003e6d63e92,Moviepilot,web,2007.0,7000000.0,Berlin,52.494727,13.398504,"{'type': 'Point', 'coordinates': [13.398504, 5...",1.758731
0,6,14,5e4179ba353fb003e6d635ce,99designs,design,2008.0,35000000.0,Berlin,52.498620,13.446903,"{'type': 'Point', 'coordinates': [13.4469031, ...",2.054640


In [34]:
u.calculate_distance(office, [design])
design

,index,_id,name,category_code,founded_year,Total_funding,city,latitude,longitude,location,distance
0,14,5e4179ba353fb003e6d635ce,99designs,design,2008.0,35000000.0,Berlin,52.49862,13.446903,"{'type': 'Point', 'coordinates': [13.4469031, ...",2.05464


## Vamos a representarlo en un mapa

In [35]:
berlin = u.geocode("Berlin")
office = {"lat": office["latitude"][0], "long": office["longitude"][0]}

In [38]:
mapa = u.locate(berlin, office, [vegan, bar, starbucks, kindergarten], [arp_berlin, companies_berlin_1M])
mapa

In [37]:
mapa.save('OUTPUT/mapa')